In [158]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import statsmodels.api as sm
import numpy as np
import geopandas as gpd
from ipfn import ipfn
from matplotlib import pyplot as plt
from haversine import haversine_vector, Unit
pd.options.mode.chained_assignment = None  # default='warn'
from sttn.data.lehd import OriginDestinationEmploymentDataProvider
from sklearn.metrics import mean_squared_error

provider = OriginDestinationEmploymentDataProvider()

import math
from sttn.network import SpatioTemporalNetwork
from sttn.utils import add_distance

%matplotlib inline

In [26]:
cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]

## Example: unconstrained log-MSE gravity model

In [61]:
def predict_unconstrained_log_mse(edges):
    '''
    Input: Pandas dataframe with origin, destination, distance, jobs, residence and flow columns
    Output; dataframe with with model predictions in 'predicted_flow' column
    '''
    copy = edges.copy()
    # replace zero values with 1
    copy.loc[(copy['jobs'] == 0), 'jobs'] = 1 
    copy.loc[(copy['residence'] == 0), 'residence'] = 1
    
    X = sm.add_constant(np.log(copy.distance))
    y = np.log(edges['flow']) - np.log(copy.jobs) - np.log(copy.residence)
    
    model = sm.OLS(y, X).fit()
    prediction = model.predict(X)
    copy['predicted_flow'] = np.exp(prediction + np.log(copy.jobs) + np.log(copy.residence))
    
    return copy[['origin', 'destination', 'flow', 'predicted_flow']]

## Mingyi and Devashish, please add your models below:

In [72]:
# prediction functions that take a Pandas dataframe with origin, destination, distance, jobs, residence and flow columns
# and return a dataframe with 'origin', 'destination', 'flow', 'predicted_flow' columns
state_network = provider.get_data(state='ny', year=2018)
city_network = state_network.filter_nodes(state_network.nodes.county.isin(['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']))


In [149]:
def build_training_set(network, target_column):
    # explode the dataset to include all node pairs
    node_ids = network.nodes.index.values
    origins = pd.DataFrame(node_ids, columns = ['origin'])
    destinations = pd.DataFrame(node_ids, columns = ['destination'])
    cartesian_product = origins.merge(destinations, how='cross')
    
    # compute distnace between all pairs
    centroid = network.nodes.centroid
    centroid_long = centroid.x
    centroid_long.name = 'long'
    centroid_lat = centroid.y
    centroid_lat.name = 'lat'
    centroids = pd.concat([centroid_long, centroid_lat], axis=1)
    centroid_from = cartesian_product.join(centroids, on=network._origin).rename(columns={'long': 'long_from', 'lat': 'lat_from'})
    centroid_all = centroid_from.join(centroids, on=network._destination).rename(columns={'long': 'long_to', 'lat': 'lat_to'})
    from_points = list(zip(centroid_all.lat_from, centroid_all.long_from))
    to_points = list(zip(centroid_all.lat_to, centroid_all.long_to))
    centroid_all['distance'] = haversine_vector(from_points, to_points, Unit.KILOMETERS)
    centroid_all.drop(['long_from', 'lat_from', 'long_to', 'lat_to'], axis=1, inplace=True)
    centroid_all.loc[centroid_all.distance == 0, 'distance'] = 0.2
    
    # compute jobs and residence
    comp_aggs={target_column: 'sum'}
    jobs = city_network.agg_adjacent_edges(aggs=comp_aggs, outgoing=False).rename(columns={target_column: 'jobs'})
    residence = city_network.agg_adjacent_edges(aggs=comp_aggs, outgoing=True).rename(columns={target_column: 'residence'})
    features = centroid_all.join(residence, on='origin').join(jobs, on='destination')
    
    # merge flow data
    flow = network.edges.rename(columns={target_column: 'flow'})[['origin', 'destination', 'flow']]
    combined = features.merge(flow, how='left', on=['origin', 'destination']).fillna(0)
    
    return combined

In [160]:
prediction = {}
target_columns = ['S000'] #['SE01', 'SE02', 'SE03']

for city, state, conties in cities:
    state_network = provider.get_data(state=state, year=2018)
    city_network = state_network.filter_nodes(state_network.nodes.county.isin(conties))
    
    
    for target_column in target_columns:
        training_set = build_training_set(city_network, target_column)
        # remove for liner MSE models
        training_set.loc[training_set['flow'] == 0, 'flow'] = 0.1

        predicted = predict_unconstrained_log_mse(training_set)
        assert training_set.shape[0] == predicted.shape[0], f'Size does not match'
        mse = mean_squared_error(predicted.flow, predicted.predicted_flow)
        print(f"Processing {city}, MSE: {mse}")
        
        prediction[(city, target_column)] = predicted

<ipython-input-149-fe05c9531592>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = network.nodes.centroid


Processing New York City, MSE: 4.572668445420518


<ipython-input-149-fe05c9531592>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = network.nodes.centroid


Processing Los Angeles, MSE: 6.457875692158243


<ipython-input-149-fe05c9531592>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = network.nodes.centroid


...

<ipython-input-149-fe05c9531592>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = network.nodes.centroid


Processing Dallas, MSE: 55.38068979821111


<ipython-input-149-fe05c9531592>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = network.nodes.centroid


Processing San Jose, MSE: 115.03088066119258
Processing Austin, MSE: 282.93737900345224


<ipython-input-149-fe05c9531592>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = network.nodes.centroid


In [164]:
predicted = prediction[('New York City','S000')]
predicted

,origin,destination,flow,predicted_flow
0,36081046200,36081046200,36.0,2.098990
1,36081046200,36081045000,9.0,0.175147
2,36081046200,36081045400,11.0,1.401250
3,36081046200,36081045500,1.0,2.987907
4,36081046200,36081045600,1.0,0.322323
...,...,...,...,...
4652644,36061016900,36081108500,0.1,0.202966
4652645,36061016900,36081109300,0.1,0.617380
4652646,36061016900,36081141700,0.1,0.915412
4652647,36061016900,36081006202,0.1,0.495803
